In [1]:
import pandas as pd
import pickle
import numpy as np

with open( '/share/hel/datasets/jobiqo/talent.com/JobRec/unbalanced_test.pkl', 'rb') as file:
    test_dicts = pickle.load(file)
bios_test = pd.DataFrame(test_dicts).reset_index()
bios_test = bios_test.drop('index', axis=1)


with open( '/share/hel/datasets/jobiqo/talent.com/JobRec/uk_jobs.pkl', 'rb') as file:
    dicts = pickle.load(file)
uk_jobs = pd.DataFrame(dicts).reset_index()
uk_jobs = uk_jobs.drop('index', axis=1)

In [2]:
base_path = '/share/rk8/home/deepak/JobAd_Rec/Models/'

Avg Neutrality score @10

In [3]:
def get_neutrality(file):
    neutrality_score = 0 
    for i in file:
        neutrality_score = neutrality_score + uk_jobs.iloc[i['corpus_id'][:10]]['neutrality'].mean()
    return neutrality_score/len(file)

In [19]:
import os

result = {}
for model in os.listdir(base_path):
    if 'shahed_result.pkl' in model:
        with open(base_path+model, 'rb') as file:
            result[model.replace('shahed_result.pkl','')] = get_neutrality(pickle.load(file))

In [20]:
new_result = {'0.0':[], '0.1':[], '0.2':[], '0.3':[], '0.4':[],'0.6':[],'0.8':[], '1.0':[]}


for key in result.keys():
    for new_key in new_result.keys():
        if new_key in key:
            new_result[new_key].append(result[key])

In [21]:
for key in new_result.keys():
    print((key, sum(new_result[key][1:])/len(new_result[key][1:])))

('0.0', 0.9531080070241037)
('0.1', 0.9546807154010772)
('0.2', 0.9544876859445962)
('0.3', 0.9575129559801275)
('0.4', 0.9561346544970625)
('0.6', 0.9576823867542487)
('0.8', 0.9613789789099404)
('1.0', 0.9775374115320563)


nDCG

In [3]:
from sklearn.metrics import ndcg_score

def get_ndcg(dicts):
    ndcg = []
    for i in range(len(bios_test)):
        ndcg.append(ndcg_score(np.asarray([uk_jobs.iloc[dicts[i]['corpus_id']]['title'].apply(lambda x: 1 if x==bios_test['raw_title'][i] else 0)]),[dicts[i]['scores']],k=10))
    
    return sum(ndcg)/len(ndcg)

In [4]:
Models = ['1500_distilroberta-base_2024-01-25_02-54-56','1390_distilroberta-base_2024-01-25_02-54-56','5915_distilroberta-base_2024-01-25_02-54-56']

result = []

for model in Models:
    with open(base_path+model+'result.pkl', 'rb') as file:
        result.append(get_ndcg(pickle.load(file)))

print('Baseline nDCG is {:.3f} with std {:.3f}'.format(np.mean(np.array(result)),np.std(np.array(result))))


Baseline nDCG is 0.745 with std 0.014


In [5]:
Models = ['1500_distilroberta-base_2024-01-25_02-54-56','1390_distilroberta-base_2024-01-25_02-54-56','5915_distilroberta-base_2024-01-25_02-54-56']

result = []

for model in Models:
    with open(base_path+model+'mask_result.pkl', 'rb') as file:
        result.append(get_ndcg(pickle.load(file)))

print('Baseline with red words masked during testing gives nDCG is {:.3f} with std {:.3f}'.format(np.mean(np.array(result)),np.std(np.array(result))))

Baseline with red words masked during testing gives nDCG is 0.633 with std 0.005


nDCG separate

In [6]:
import numpy as np
from sklearn.metrics import ndcg_score

def get_ndcg_separate(dicts):
    ndcg = []
    for i in range(len(bios_test)):
        ndcg.append(ndcg_score(np.asarray([uk_jobs.iloc[dicts[i]['corpus_id']]['title'].apply(lambda x: 1 if x==bios_test['raw_title'][i] else 0)]),[dicts[i]['scores']],k=10))
    

    ndcg = [ndcg[i] for i in bios_test[bios_test.gender =='M'].index]
    female_ndcg = [ndcg[i] for i in bios_test[bios_test.gender =='F'].index]
    
    if len(male_ndcg)>0:
        male_ndcg =sum(male_ndcg)/len(male_ndcg)
    else:
        male_ndcg = 0
    if len(female_ndcg)>0:
        female_ndcg =sum(female_ndcg)/len(female_ndcg)
    else:
        female_ndcg = 0
     
    return (male_ndcg, female_ndcg)

In [7]:
Models = ['1500_distilroberta-base_2024-01-25_02-54-56','1390_distilroberta-base_2024-01-25_02-54-56','5915_distilroberta-base_2024-01-25_02-54-56']

result_male = []
result_female = []

for model in Models:
    with open(base_path+model+'result.pkl', 'rb') as file:
        m,f = get_ndcg_separate(pickle.load(file))
        result_male.append(m)
        result_female.append(f)



print("Baseline male nDCG is {:.3f} with std {:.3f}".format(np.mean(np.array(result_male)),np.std(np.array(result_male))))
print("Baseline female nDCG is {:.3f} with std {:.3f}".format(np.mean(np.array(result_female)),np.std(np.array(result_female))))
print('The gap is {:.3f}'.format(abs(np.mean(np.array(result_male))-np.mean(np.array(result_female)))))

Baseline male nDCG is 0.737 with std 0.016
Baseline female nDCG is 0.752 with std 0.014
The gap is 0.015


In [8]:
Models = ['1500_distilroberta-base_2024-01-25_02-54-56','1390_distilroberta-base_2024-01-25_02-54-56','5915_distilroberta-base_2024-01-25_02-54-56']

result_male = []
result_female = []

for model in Models:
    with open(base_path+model+'mask_result.pkl', 'rb') as file:
        m,f = get_ndcg_separate(pickle.load(file))
        result_male.append(m)
        result_female.append(f)



print("Baseline with red words masked during testing gives male nDCG is {:.3f} with std {:.3f}".format(np.mean(np.array(result_male)),np.std(np.array(result_male))))
print("Baseline with red words masked during testing gives female nDCG is {:.3f} with std {:.3f}".format(np.mean(np.array(result_female)),np.std(np.array(result_female))))
print('The gap is {:.3f}'.format(abs(np.mean(np.array(result_male))-np.mean(np.array(result_female)))))

Baseline with red words masked during testing gives male nDCG is 0.642 with std 0.003
Baseline with red words masked during testing gives female nDCG is 0.624 with std 0.008
The gap is 0.018


Testing Counterfactual

In [ ]:
# Given the input create counterfactual and get the ranking. You have 2 set of ranking original and couterfactual. Find original vs counterfactual list difference.

In [1]:
#Get counterfactual ranking 
import os
from testing import testing

device=5
pth = '/'

base_path = '/share/rk8/home/deepak/JobAd_Rec/Models/'

for alpha in ['0.0', '0.1', '0.2', '0.3', '0.4']:
    for model in os.listdir(base_path):
        if alpha in model and '02-20' in model and 'shahed_result.pkl' not in model:
            model_path = base_path+model
            try:
                testing(path=model_path,gpu=device,counterfactual=True)
            except:
                pass

/home/deepak/anaconda3/envs/evonlp/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
100%|██████████| 560/560 [56:12<00:00,  6.02s/it]


In [5]:
bios_test

,raw,raw_title,gender,bio,counter_bio
0,Klotilda Lala is a staff _ with MRPR and has b...,accountant,F,Alice is a staff _ with MRPR and has been with...,Bob is a staff _ with MRPR and has been with t...
1,"Sandy Nicholson is a _ based in Toronto, Canad...",photographer,M,"Bob is a _ based in Toronto, Canada. Sandy’s p...","Alice is a _ based in Toronto, Canada. Sandy’s..."
2,Susan O'Connell is a Principal Architect for R...,architect,F,"Alice is a Principal _ for RevGen Partners, le...","Bob is a Principal _ for RevGen Partners, lead..."
3,Ashley Ritzo is a registered/licensed _ with a...,dietitian,F,Alice is a registered/licensed _ with a B.S. i...,Bob is a registered/licensed _ with a B.S. in ...
4,Jeramie J. Perez is a senior _ in the emerging...,paralegal,M,Bob is a senior _ in the emerging companies pr...,Alice is a senior _ in the emerging companies ...
...,...,...,...,...,...
555,Emily Sayers Slocum is a _-midwife and Doctors...,nurse,F,Alice is a _-midwife and Doctors Without Borde...,Bob is a _-midwife and Doctors Without Borders...
556,Kara K Sharpf is a Nurse Practitioner Speciali...,nurse,F,Kara K Alice is a _ Practitioner Specialist in...,Kara K Bob is a _ Practitioner Specialist in D...
557,"Shawn Moe is a _ at the IBM lab in Lenexa, Kan...",software engineer,M,"Bob is a _ at the IBM lab in Lenexa, Kansas. H...","Alice is a _ at the IBM lab in Lenexa, Kansas...."
558,Mrs. Joscelyn Vanduren is a female Nurse Pract...,nurse,F,Mrs. Alice is a female _ Practitioner (taxonom...,Mrs. Bob is a female _ Practitioner (taxonomy ...


nDCG gap over counterfactual

In [ ]:
import numpy as np
from sklearn.metrics import ndcg_score

def get_ndcg_separate(dicts,dicts_counter):
    ndcg = []
    ndcg_counter = []
    for i in range(len(bios_test)):
        ndcg.append(ndcg_score(np.asarray([uk_jobs.iloc[dicts[i]['corpus_id']]['title'].apply(lambda x: 1 if x==bios_test['raw_title'][i] else 0)]),[dicts[i]['scores']],k=10))
        ndcg_counter.append(ndcg_score(np.asarray([uk_jobs.iloc[dicts_counter[i]['corpus_id']]['title'].apply(lambda x: 1 if x==bios_test['raw_title'][i] else 0)]),[dicts_counter[i]['scores']],k=10))

    

    ndcg_separation = [abs(ndcg[i]-ndcg_counter[i]) for i in range(len(bios_test))]
        
    if len(ndcg)>0:
        ndcg =sum(ndcg)/len(ndcg)
    else:
        ndcg = 0
    if len(ndcg_counter)>0:
        ndcg_counter =sum(ndcg_counter)/len(ndcg_counter)
    else:
        ndcg_counter = 0
    if len(ndcg_separation)>0:
        ndcg_separation =sum(ndcg_separation)/len(ndcg_separation)
    else:
        ndcg_separation = 0 
    return (ndcg, ndcg_counter,ndcg_separation)

In [ ]:
import os

result = {}
for model in os.listdir(base_path):
    if '02-20' in model and 'shahed_result.pkl' in model:
        model_path = base_path+model
        
        with open( model_path,'rb') as file1:
            
            with open( model_path.replace('shahed','counter'),'rb') as file2:
                result[model.replace('shahed_result.pkl','')] = get_ndcg_separate(pickle.load(file1),pickle.load(file2))[-1]


new_result = {'0.0':[], '0.2':[], '0.4':[],'0.6':[],'0.8':[], '1.0':[]}


for key in result.keys():
    for new_key in new_result.keys():
        if new_key in key:
            new_result[new_key].append(result[key])

for key in new_result.keys():
    print((key, sum(new_result[key])/len(new_result[key]),len(new_result[key])))

('0.0', 0.026477710728856508, 3)
('0.2', 0.035883420193732904, 3)
('0.4', 0.05261764709776821, 3)
('0.6', 0.02923345778491597, 3)
('0.8', 0.012060492937521424, 2)
('1.0', 0.00836899053686995, 3)


In [ ]:
for key in new_result.keys():
    print(sum(new_result[key])/len(new_result[key]))

0.026477710728856508
0.035883420193732904
0.05261764709776821
0.02923345778491597
0.012060492937521424
0.00836899053686995


SDR

In [ ]:
def SDR(dicts):
    male_item_ids=[]
    female_item_ids=[]
    for male in [dicts[i] for i in bios_test[bios_test.gender =='M'].index]:
        male_df = pd.DataFrame(male)
        male_item_ids = male_item_ids+list(male_df[male_df['scores']>sorted(male_df['scores'],reverse=True)[10]]['corpus_id'])
    
    for female in [dicts[i] for i in bios_test[bios_test.gender =='F'].index]:
        female_df = pd.DataFrame(female)
        female_item_ids = female_item_ids+list(female_df[female_df['scores']>sorted(female_df['scores'],reverse=True)[10]]['corpus_id'])

    output = (len(male_item_ids)+len(female_item_ids)-2*len(set(male_item_ids).intersection(female_item_ids)))/(len(male_item_ids)+len(female_item_ids)-len(set(male_item_ids).intersection(female_item_ids)))
    #output = len(set(male_item_ids).intersection(female_item_ids))
    return output

In [ ]:
import os

result = {}
for model in os.listdir(base_path):
    if '02-20' in model and 'shahed_result.pkl' in model:
        with open(base_path+model, 'rb') as file:
            result[model.replace('shahed_result.pkl','')] = SDR(pickle.load(file))

In [ ]:
new_result = {'0.0':[], '0.2':[], '0.4':[],'0.6':[],'0.8':[], '1.0':[]}


for key in result.keys():
    for new_key in new_result.keys():
        if new_key in key:
            new_result[new_key].append(result[key])

In [ ]:
for key in new_result.keys():
    print((key, sum(new_result[key])/len(new_result[key])))

('0.0', 0.9098421606837759)
('0.2', 0.9109633927218549)
('0.4', 0.9116280081505629)
('0.6', 0.929333368271465)
('0.8', 0.9795598465644189)
('1.0', 0.9828790894340358)


LDR

In [ ]:
import numpy as np
from sklearn.metrics import ndcg_score

def LDR(dicts,dicts_counter):
    output = []
    
    for i in range(len(bios_test)):
        output.append((sum(((dicts[i]['corpus_id']!=dicts_counter[i]['corpus_id'])*1)[:10]))/10)
        #print(((dicts[i]['corpus_id']!=dicts_counter[i]['corpus_id'])*1)[:10])
        #break
           
    if len(output)>0:
        ldr =sum(output)/len(output)
    else:
        ldr = 0
     
    return ldr

In [ ]:
import os

result = {}
for model in os.listdir(base_path):
    if '02-20' in model and 'shahed_result.pkl' in model:
        model_path = base_path+model
        
        with open( model_path,'rb') as file1:
            
            with open( model_path.replace('shahed','counter'),'rb') as file2:
                result[model.replace('shahed_result.pkl','')] = LDR(pickle.load(file1),pickle.load(file2))


new_result = {'0.0':[], '0.2':[], '0.4':[],'0.6':[],'0.8':[], '1.0':[]}


for key in result.keys():
    for new_key in new_result.keys():
        if new_key in key:
            new_result[new_key].append(result[key])

for key in new_result.keys():
    print((key, sum(new_result[key])/len(new_result[key]),len(new_result[key])))

('0.0', 0.6904166666666663, 3)
('0.2', 0.6778571428571422, 3)
('0.4', 0.6302976190476185, 3)
('0.6', 0.48666666666666664, 3)
('0.8', 0.5874107142857145, 2)
('1.0', 0.42988095238095236, 3)


In [ ]:
for key in new_result.keys():
    print( sum(new_result[key])/len(new_result[key]))

0.6904166666666663
0.6778571428571422
0.6302976190476185
0.48666666666666664
0.5874107142857145
0.42988095238095236


EMD

In [ ]:
from PyEMD import EMD
emd = EMD()


MWU test

Who is higher?